In [1]:
import pandas as pd
import mysql.connector
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Membaca data latih dari file Excel
data_latih = pd.read_excel('data-train.xlsx')

# Membagi data latih menjadi fitur dan label
X = data_latih.drop('Tipe', axis=1)
y = data_latih['Tipe']

# Melakukan preprocessing pada data
sc = StandardScaler()
X = sc.fit_transform(X)

# Melatih model SVM dengan seluruh data latih
model = SVC(C=1, gamma=0.01, kernel='linear', random_state=42)
model.fit(X, y)

# Create a connection to the MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db_nentuin"
)

# Read data for hasil_tertinggi from the table
query = "SELECT V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, K1, K2, K3, K4, K5, K6, K7, K8, K9, K10, K11, K12 FROM hasil_pilihan"
data_hasil_tertinggi = pd.read_sql(query, connection)

# Close the connection to the database
connection.close()

# Preprocess the data
data_hasil_tertinggi = data_hasil_tertinggi.replace({"Sering": 0, "Kadang-kadang": 1, "Jarang": 2})
data_hasil_tertinggi = sc.transform(data_hasil_tertinggi)

# Memprediksi kelas hasil tertinggi
hasil_tertinggi = model.predict(data_hasil_tertinggi)

# Membuat dictionary untuk mengubah angka ke label kelas
kelas_dict = {0: 'Visual', 1: 'Auditori', 2: 'Kinestetik'}

# Mengubah hasil prediksi menjadi label kelas
hasil_tertinggi_label = [kelas_dict[kelas] for kelas in hasil_tertinggi]

print("Kelas hasil tertinggi: ", hasil_tertinggi_label)

# Melakukan cross-validation dengan 10 folds
scores = cross_val_score(model, X, y, cv=10)

# Mencetak akurasi rata-rata dan standar deviasi dari cross-validation
print("Akurasi rata-rata: %0.2f%% (+/- %0.2f%%)" % (scores.mean()*100, scores.std() * 200))

# Create a connection to the MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db_nentuin"
)

# Membuat cursor untuk menjalankan query
cursor = connection.cursor()

# Query untuk menyimpan hasil kelas tertinggi dan akurasi rata-rata ke tabel "hasil"
insert_query = "INSERT INTO hasil (hasil, akurasi) VALUES (%s, %s)"
values = (', '.join(hasil_tertinggi_label), scores.mean())

# Menjalankan query INSERT
cursor.execute(insert_query, values)

# Melakukan commit perubahan ke database
connection.commit()

# Menutup cursor dan koneksi ke database
cursor.close()
connection.close()

print("Data berhasil disimpan ke tabel 'hasil'.")

Kelas hasil tertinggi:  ['Auditori']
Akurasi rata-rata: 80.83% (+/- 17.59%)
